In [1]:
# https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

# https://github.com/williamscott701/Information-Retrieval/blob/master/2.%20TF-IDF%20Ranking%20-%20Cosine%20Similarity%2C%20Matching%20Score/TF-IDF.ipynb

In [2]:
cd Desktop

In [3]:
cd Stemaway ML Workspace

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from nltk.tokenize import RegexpTokenizer

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

# %load_ext autotime

In [5]:
# Creating dataframe

dtf = pd.read_csv('ML Team 6 Data Processing - data cleaning.csv')
dtf.info()
dtf = dtf.drop_duplicates(subset=['Topic'], keep='first').reset_index(drop=True)
dtf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221909 entries, 0 to 221908
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Unnamed: 0        221909 non-null  int64 
 1   Topic             221909 non-null  object
 2   Title Keywords    221902 non-null  object
 3   Content Keywords  221904 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170425 entries, 0 to 170424
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Unnamed: 0        170425 non-null  int64 
 1   Topic             170425 non-null  object
 2   Title Keywords    170419 non-null  object
 3   Content Keywords  170422 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.2+ MB


In [6]:
dtf.head()

,Unnamed: 0,Topic,Title Keywords,Content Keywords
0,0,Restrictons of Python compared to Ruby: lambda's,lambda restrictons python compared ruby,expression restricted good quote would like kn...
1,1,Scale image in matplotlib without changing the...,matplotlib without changing scale image axis,gui imshow display myimage plot using displaye...
2,2,UnboundLocalError on nested module reimport,unboundlocalerror nested module reimport,unboundlocalerror python import sys def foo pr...
3,3,How to convert 3D RGB label image (in semantic...,semantic segmentation d gray image class index...,rgb value respectively value exists class map ...
4,4,Python minidom/xml : How to set node text with...,minidom api set node text python xml,xml tag like currently file trying load somete...


In [7]:
# Tokenization

tokenizer = RegexpTokenizer(r'\w+')
dtf['Title Keywords'] = dtf['Title Keywords'].apply(lambda x: tokenizer.tokenize(str(x)))
dtf['Content Keywords'] = dtf['Content Keywords'].apply(lambda x: tokenizer.tokenize(str(x)))

In [8]:
dtf.head()

,Unnamed: 0,Topic,Title Keywords,Content Keywords
0,0,Restrictons of Python compared to Ruby: lambda's,"[lambda, restrictons, python, compared, ruby]","[expression, restricted, good, quote, would, l..."
1,1,Scale image in matplotlib without changing the...,"[matplotlib, without, changing, scale, image, ...","[gui, imshow, display, myimage, plot, using, d..."
2,2,UnboundLocalError on nested module reimport,"[unboundlocalerror, nested, module, reimport]","[unboundlocalerror, python, import, sys, def, ..."
3,3,How to convert 3D RGB label image (in semantic...,"[semantic, segmentation, d, gray, image, class...","[rgb, value, respectively, value, exists, clas..."
4,4,Python minidom/xml : How to set node text with...,"[minidom, api, set, node, text, python, xml]","[xml, tag, like, currently, file, trying, load..."


In [9]:
# Dictionary for counting all unique words

N = len(dtf['Topic'])

contents = dtf['Content Keywords']
titles = dtf['Title Keywords']

DF = {}

for i in range(N):
    tokens = contents[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = titles[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [10]:
# Each unique word and the total count

DF

{'expression': 1346,
 'restricted': 66,
 'good': 2796,
 'quote': 550,
 'would': 13896,
 'like': 29178,
 'know': 9783,
 'wikivs': 1,
 'python': 26254,
 'cannot': 3152,
 'contain': 1071,
 'statement': 2782,
 'going': 1507,
 'page': 10133,
 'lambda': 378,
 'restrictons': 1,
 'compared': 228,
 'ruby': 620,
 'gui': 479,
 'imshow': 105,
 'display': 4093,
 'myimage': 29,
 'plot': 1652,
 'using': 40474,
 'displayed': 686,
 'want': 24971,
 'myaxeimage': 1,
 'axpect': 1,
 'auto': 1460,
 'extent': 46,
 'existing': 1359,
 'image': 6990,
 'myaxe': 1,
 'fit': 994,
 'matplotlib': 1713,
 'without': 6186,
 'changing': 1061,
 'scale': 636,
 'axis': 578,
 'unboundlocalerror': 19,
 'import': 6920,
 'sys': 685,
 'def': 3031,
 'foo': 2240,
 'print': 3649,
 'traceback': 532,
 'already': 2360,
 'imported': 295,
 'module': 4372,
 'get': 22375,
 'bin': 844,
 'usr': 630,
 'reimport': 4,
 'minimal': 187,
 'example': 13708,
 'nested': 1240,
 'rgb': 193,
 'value': 14494,
 'respectively': 102,
 'exists': 1214,
 'cla

In [11]:
# Total number of unique words

total_vocab_size = len(DF)
total_vocab_size

75494

In [12]:
total_vocab = [x for x in DF]
total_vocab[:20]

['expression',
 'restricted',
 'good',
 'quote',
 'would',
 'like',
 'know',
 'wikivs',
 'python',
 'cannot',
 'contain',
 'statement',
 'going',
 'page',
 'lambda',
 'restrictons',
 'compared',
 'ruby',
 'gui',
 'imshow']

In [13]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [14]:
# Each document's title keywords and that keyword's tf idf score

doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = contents[i]
    
    counter = Counter(tokens + titles[i])
    words_count = len(tokens + titles[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [15]:
tf_idf

{(0, 'cannot'): 0.1994973402176221,
 (0, 'contain'): 0.25343875611333777,
 (0, 'expression'): 0.24202106437435697,
 (0, 'going'): 0.2363758457664801,
 (0, 'good'): 0.20548768853477717,
 (0, 'know'): 0.14287763932652475,
 (0, 'lambda'): 0.6108520258815077,
 (0, 'like'): 0.0882425956227968,
 (0, 'page'): 0.14112025389308241,
 (0, 'python'): 0.18704447378255437,
 (0, 'quote'): 0.2867160827372294,
 (0, 'restricted'): 0.39206819222532685,
 (0, 'statement'): 0.20573858511858065,
 (0, 'wikivs'): 0.5676454641668779,
 (0, 'would'): 0.12533140977054605,
 (1, 'auto'): 0.2163263660066729,
 (1, 'axpect'): 0.5160413310607981,
 (1, 'display'): 0.16948994080346674,
 (1, 'displayed'): 0.25062373507614333,
 (1, 'existing'): 0.21958256750760935,
 (1, 'extent'): 0.37254131191761114,
 (1, 'fit'): 0.23378698757904137,
 (1, 'gui'): 0.2669213799997985,
 (1, 'image'): 0.2903343252168653,
 (1, 'imshow'): 0.3355735168084289,
 (1, 'myaxe'): 0.5160413310607981,
 (1, 'myaxeimage'): 0.5160413310607981,
 (1, 'myimage

In [16]:
# Each document's content keywords and that keyword's tf idf score

doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = titles[i]
    counter = Counter(tokens + contents[i])
    words_count = len(tokens + contents[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [17]:
tf_idf_title

{(0, 'compared'): 0.3306167230171632,
 (0, 'lambda'): 0.6108520258815077,
 (0, 'python'): 0.18704447378255437,
 (0, 'restrictons'): 0.5676454641668779,
 (0, 'ruby'): 0.2807362690982012,
 (1, 'axis'): 0.25839790846929583,
 (1, 'changing'): 0.23082487554980086,
 (1, 'image'): 0.2903343252168653,
 (1, 'matplotlib'): 0.20906688021544448,
 (1, 'scale'): 0.25405849128751545,
 (1, 'without'): 0.15072049401665974,
 (2, 'module'): 0.3488430392842459,
 (2, 'nested'): 0.23439922279489978,
 (2, 'reimport'): 0.993963671567943,
 (2, 'unboundlocalerror'): 0.8619356371755724,
 (3, 'class'): 0.2314067755167023,
 (3, 'convert'): 0.15425112047876893,
 (3, 'd'): 0.3710148849410321,
 (3, 'gray'): 0.30244019671314404,
 (3, 'image'): 0.13306989905772992,
 (3, 'index'): 0.15827393581715876,
 (3, 'label'): 0.4093967090834528,
 (3, 'rgb'): 0.5648498587361812,
 (3, 'segmentation'): 0.6418542535036516,
 (3, 'semantic'): 0.6021479081910388,
 (3, 'start'): 0.1682094074511572,
 (4, 'api'): 0.154330466690163,
 (4, 'm

In [18]:
tf_idf[(0, 'python')]

0.18704447378255437

In [19]:
tf_idf_title[(0, 'python')]

0.18704447378255437

In [20]:
alpha = 0.3
for i in tf_idf:
    tf_idf[i] *= alpha

In [21]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [22]:
len(tf_idf)

3227544

In [23]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [24]:
def matching_score(k, query):
    preprocessed_query = (remove_stop_words(query.lower()))
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    
    print("")
    
    for i in query_weights[:k]:
        rec = dtf['Topic'][i[0]]
        if rec == query:
            print('Duplicate Topic:  ' + rec + '   ' + str(i))
        else:
            print('Recommended Topic:  ' + rec + '   ' + str(i))

In [27]:
matching_score(10, 'javascript dot notation like square bracket use variable example using call associative array create understand allows')

# Joining arrays within group by clause
# How to save a file downloaded from requests to another directory?

Matching Score

Query: javascript dot notation like square bracket use variable example using call associative array create understand allows

['javascript', 'dot', 'notation', 'like', 'square', 'bracket', 'use', 'variable', 'example', 'using', 'call', 'associative', 'array', 'create', 'understand', 'allows']

Recommended Topic:  How to use variables in dot notation like square bracket notation   (48872, 3.1631171096466577)
Recommended Topic:  Does MySQL allows to create database with dot?   (112384, 2.416492910251931)
Recommended Topic:  How to get array of values from an associative arrays?   (15086, 2.1274550893455713)
Recommended Topic:  What do square brackets mean in pip install?   (152914, 1.8278369220580437)
Recommended Topic:  Static variables in JavaScript   (169251, 1.8079750812295874)
Recommended Topic:  What's the difference between the square bracket and dot notations in Python?   (129324, 1.80493406390163)
Recommended Topic:  How to use a link to call JavaScript?   (1621